### Формирование модели машинного обучения

Текущий ноутбук рассматривает анализ датасета [Auto MPG](https://archive.ics.uci.edu/ml/datasets/Auto+MPG).

В работе рассматривается:
1. обучение регрессионной модели для определения потребления автомобилями топлива в городском цикле в галлонах на милю по представленным данным;
2. сохранение обученной модели в виде **конвейера обработки данных** для последующего использования.

In [ ]:
import numpy as np
import pandas as pd
from seaborn import pairplot
from sklearn.compose import ColumnTransformer, TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split, GridSearchCV
from pickle import dump
from pathlib import Path

In [ ]:
col_names = ("mpg", "cylinders", "displacement", "horsepower", "weight", 
             "acceleration", "model_year", "origin", "car name")
df = pd.read_csv("../data/auto-mpg-data.tsv", 
                 delim_whitespace=True,
                 names=col_names,
                 header=None)

In [ ]:
df.head()

In [ ]:
df.info()

Столбец "`horsepower`" содержит что-то, кроме чисел. Выясним, что.

In [ ]:
df["horsepower_floats"] = pd.to_numeric(df["horsepower"], errors='coerce')

In [ ]:
df.info()

In [ ]:
df[df["horsepower_floats"].isna()]

Итак, столбец "`horsepower`" содержит пропуски, обозначенные знаком вопроса. Заменим их на пустые значения, удалим вспомогательный столбец и колонку "`car name`", которая не используется для анализа.

In [ ]:
df["horsepower"] = df["horsepower_floats"]
df = df.drop("horsepower_floats", axis=1)
df = df.drop("car name", axis=1)

In [ ]:
df.info()

In [ ]:
df

Посмотрим на распределение данных. Какие выводы по ним можно сделать?

In [ ]:
pairplot(df)

Посмторим на распределение категориальных переменных.

In [ ]:
df["cylinders"].value_counts()

In [ ]:
df["origin"].value_counts()

Выясним, содержатся ли выбросы в данных.

In [ ]:
df[["mpg", "displacement", "horsepower", "weight",
    "acceleration", "model_year"]].boxplot(rot=45)

Выбросы есть, но их немного, и они не критичные. Убирать их не будем.

Сформируем конвейеры для обработки количественных и категориальных данных.

In [ ]:
numeric_features = ["displacement", "horsepower", "weight", 
                    "acceleration", "model_year"]
numeric_transformer = Pipeline(
    steps=[
        ("imputer", SimpleImputer(strategy="median")), 
        ("scaler", StandardScaler()),
    ]
)

In [ ]:
categorical_features = ["cylinders", "origin"]
categorical_transformer = Pipeline(
    steps=[
        ("onehot-encoder", OneHotEncoder(drop="first", 
                                         handle_unknown="ignore")),
    ]
)

In [ ]:
preprocessor = ColumnTransformer(
    transformers=[
        ("numeric", numeric_transformer, numeric_features),
        ("categorical", categorical_transformer, categorical_features),
    ]
)

In [ ]:
regressor = Ridge()
tr_regressor = TransformedTargetRegressor(regressor=regressor,
                                          transformer=MinMaxScaler())
pipe = Pipeline(
    steps=[
        ("preprocessor", preprocessor), 
        ("regressor", tr_regressor)
    ]
)

In [ ]:
train, test = train_test_split(df, test_size=0.3)

In [ ]:
train.head()

In [ ]:
X_train = train[["cylinders", 
                 "displacement", 
                 "horsepower", 
                 "weight", 
                 "acceleration", 
                 "model_year", 
                 "origin",
                ]]
Y_train = train[["mpg"]]

In [ ]:
X_test = test[["cylinders", 
               "displacement", 
               "horsepower", 
               "weight", 
               "acceleration", 
               "model_year", 
               "origin",
              ]]
Y_test = test[["mpg"]]

Подберем коэффициент регуляризации для линейной регрессии.

In [ ]:
param_grid = {
    "regressor__regressor__alpha": np.linspace(0.0, 3.0, num=31),
}

In [ ]:
search_cv = GridSearchCV(pipe, param_grid)

In [ ]:
search_cv.fit(X_train, Y_train)

In [ ]:
print("Best params:")
print(search_cv.best_params_)

In [ ]:
search_cv.score(X_train, Y_train)

In [ ]:
search_cv.score(X_test, Y_test)

Сохраним полученный конвейер обработки данных для дальнейшего использования.

In [ ]:
Path("../models").mkdir(parents=True, exist_ok=True)

with open("../models/pipeline.pkl", "wb") as file:
    dump(search_cv, file)